# 🛩️ HMIS Data Checker
This script will compare the Guinea HMIS with Stop Palu staging workbook importer database. This is not the same thing as DHIS, just so that you know. Until HMIS data was pulled in directly to DHIS2, this staging datbase was used where workbooks were imported and then a DHIS2 import file was generated from this database. 

The purpose of this report is to document the difference between the monthly malaria data in the HMIS and the workbooks that the Stop Palu team uses.

First, loop through the same orgunit list and using this create two pandas dataframes one that is from the HMIS data and another from the worksbooks.

##  Step 0: Get everything ready to start


In [1]:
import xml.etree.ElementTree as ET
tree1 = ET.parse('orgUnits.xml')
admin1 = tree1.getroot()

orgs_mapping = []

for orgxml in admin1.iter('orgUnit'):
    
    orgs = {}
    orgs['hmisUID'] = orgxml.attrib['hmisUID']
    orgs['projectUID'] = orgxml.attrib['projectUID']
    orgs['district'] = orgxml.attrib['district']
    orgs['region'] = orgxml.attrib['region']  
    orgs['facility'] = orgxml.attrib['facility'] 
    orgs_mapping.append(orgs)
    
   # print(orgs['facility']+ " "+ orgs['region'])
    

In [2]:
len(orgs_mapping)

211

In [3]:
import csv
orgunit_csv = open('stoppalu_orgs_040819.csv', 'w')
csvwriter = csv.writer(orgunit_csv)
count = 0
for org in orgs_mapping:
    if count == 0:
        csvwriter.writerow([ 'name', 'district', 'region'])
        count = count + 1
        
    csvwriter.writerow([org['facility'], org['district'], org['region'] ])
    

In [4]:
import numpy as np
import pandas as pd
import requests

pd.set_option('display.max_columns', None)


In [5]:
# variable names - FRENCH
FACILITY_ID = "Facility ID"
CONSULTED_UNDER5_ONSITE = "Patients consultés, < 5, Par la structure"
CONSULTED_UNDER5_AGENT="Patients consultés, < 5, Par les ASC"
CONSULTED_UNDER5_DEAD="Patients consultés, < 5, Total décès"
CONSULTED_5ANDOVER_ONSITE = "Patients consultés, >= 5, Par la structure"
CONSULTED_5ANDOVER_AGENT="Patients consultés, >= 5, Par les ASC"
CONSULTED_5ANDOVER_DEAD="Patients consultés, >= 5, Total décès"
CONSULTED_PREGNANT_ONSITE = "Patients consultés, Femmes enceintes, Par la structure"
CONSULTED_PREGNANT_DEAD="Patients consultés, Femmes enceintes, Total décès"

SUSPECTED_MILD_UNDER5_MALE_ONSITE = "Cas suspects, Simple, Mâle, < 5, Par la structure"
SUSPECTED_MILD_UNDER5_FEMALE_ONSITE = "Cas suspects, Simple, Femelle, < 5, Par la structure"
SUSPECTED_SEVERE_UNDER5_MALE_ONSITE = "Cas suspects, Grave, Mâle, < 5, Par la structure"
SUSPECTED_SEVERE_UNDER5_FEMALE_ONSITE = "Cas suspects, Grave, Femelle, < 5, Par la structure"
SUSPECTED_MILD_UNDER5_MALE_AGENT = "Cas suspects, Simple, Mâle, < 5, Par les ASC"
SUSPECTED_MILD_UNDER5_FEMALE_AGENT = "Cas suspects, Simple, Femelle, < 5, Par les ASC"
SUSPECTED_MILD_PREGNANT_ONSITE = "Cas suspects, Simple, Femmes enceintes, Par la structure"
SUSPECTED_SEVERE_PREGNANT_ONSITE = "Cas suspects, Grave, Femmes enceintes, Par la structure"

SUSPECTED_MILD_5ANDOVER_MALE_ONSITE = "Cas suspects, Simple, Mâle, >= 5, Par la structure"
SUSPECTED_MILD_5ANDOVER_FEMALE_ONSITE = "Cas suspects, Simple, Femelle, >= 5, Par la structure"
SUSPECTED_SEVERE_5ANDOVER_MALE_ONSITE = "Cas suspects, Grave, Mâle, >= 5, Par la structure"
SUSPECTED_SEVERE_5ANDOVER_FEMALE_ONSITE = "Cas suspects, Grave, Femelle, >= 5, Par la structure"
SUSPECTED_MILD_5ANDOVER_MALE_AGENT = "Cas suspects, Simple, Mâle, >= 5, Par les ASC"
SUSPECTED_MILD_5ANDOVER_FEMALE_AGENT = "Cas suspects, Simple, Femelle, >= 5, Par les ASC"

TESTED_MICROSCOPY_UNDER5_ONSITE = "Cas testés, Microscopy, < 5, Par la structure"
TESTED_RDT_UNDER5_ONSITE = "Cas testés, RDT, < 5, Par la structure"
TESTED_RDT_UNDER5_AGENT = "Cas testés, RDT, < 5, Par les ASC"
TESTED_MICROSCOPY_5ANDOVER_ONSITE = "Cas testés, Microscopy, >= 5, Par la structure"
TESTED_RDT_5ANDOVER_ONSITE = "Cas testés, RDT, >= 5, On Site"
TESTED_RDT_5ANDOVER_AGENT = "Cas testés, RDT, >= 5, Par les ASC"
TESTED_MICROSCOPY_PREGNANT_ONSITE = "Cas testés, Microscopy, Femmes enceintes, Par la structure"
TESTED_RDT_PREGNANT_ONSITE = "Cas testés, RDT, Femmes enceintes, Par la structure"

CONFIRMED_MICROSCOPY_UNDER5_ONSITE = "Cas confirmés , Microscopie, < 5, Par la structure"
CONFIRMED_RDT_UNDER5_ONSITE = "Cas confirmés , TDR, < 5, Par la structure"
CONFIRMED_RDT_UNDER5_AGENT = "Cas confirmés , TDR, < 5, Par les ASC"
CONFIRMED_MICROSCOPY_5ANDOVER_ONSITE = "Cas confirmés , Microscopie, >= 5, Par la structure"
CONFIRMED_RDT_5ANDOVER_ONSITE = "Cas confirmés , TDR, >= 5, Par la structure"
CONFIRMED_RDT_5ANDOVER_AGENT = "Cas confirmés , TDR, >= 5, Par les ASC"
CONFIRMED_MICROSCOPY_PREGNANT_ONSITE = "Cas confirmés , Microscopie, Femmes enceintes, Par la structure"
CONFIRMED_RDT_PREGNANT_ONSITE = "Cas confirmés , TDR, Femmes enceintes, Par la structure"

TREATED_ACT_UNDER5_MALE_ONSITE = "Cas traités avec CTA, Mâle, < 5, Par la structure"
TREATED_ACT_UNDER5_FEMALE_ONSITE = "Cas traités avec CTA, Femelle, < 5, Par la structure"
TREATED_ACT_UNDER5_MALE_AGENT = "Cas traités avec CTA, Mâle, < 5, Par les ASC"
TREATED_ACT_UNDER5_FEMALE_AGENT = "Cas traités avec CTA, Femelle, < 5, Par les ASC"
TREATED_ACT_5ANDOVER_MALE_ONSITE = "Cas traités avec CTA, Mâle, >= 5, Par la structure"
TREATED_ACT_5ANDOVER_FEMALE_ONSITE = "Cas traités avec CTA, Femelle, >= 5, Par la structure"
TREATED_ACT_5ANDOVER_MALE_AGENT = "Cas traités avec CTA, Mâle, >= 5, Par les ASC"
TREATED_ACT_5ANDOVER_FEMALE_AGENT = "Cas traités avec CTA, Femelle, >= 5, Par les ASC"
TREATED_ACT_PREGNANT_ONSITE = "Cas traités avec CTA, Femmes enceintes, Par la structure"

TREATED_SEVERE_UNDER5_MALE_ONSITE = "Cas graves traités, Mâle, < 5, Par la structure"
TREATED_SEVERE_UNDER5_FEMALE_ONSITE = "Cas graves traités, Femelle, < 5, Par la structure"
TREATED_SEVERE_5ANDOVER_MALE_ONSITE = "Cas graves traités, Mâle, >= 5, Par la structure"
TREATED_SEVERE_5ANDOVER_FEMALE_ONSITE = "Cas graves traités, Femelle, >= 5, Par la structure"
TREATED_SEVERE_PREGNANT_ONSITE = "Cas graves traités, Femmes enceintes, Par la structure"

REFERRED_UNDER5_ONSITE = "Cas référés, < 5, Par la structure"
REFERRED_UNDER5_AGENT = "Cas référés, < 5, Par les ASC"
REFERRED_5ANDOVER_ONSITE = "Cas référés, >= 5, Par la structure"
REFERRED_5ANDOVER_AGENT = "Cas référés, >= 5, Par les ASC"
REFERRED_PREGNANT_ONSITE = "Cas référés, Femmes enceintes, Par la structure"
REFERRED_PREGNANT_AGENT = "Cas référés, Femmes enceintes, Par les ASC"

NUMBER_OF_PREGNANT_WOMEN_1ST_ANC = "# total de femmes enceintes ayant fait la 1ère CPN"
NUMBER_OF_PREGNANT_1ST_DOSE_SP = "# femmes enceintes ayant reçu la 1ère dose de SP"
NUMBER_OF_PREGNANT_2ND_DOSE_SP = "Nbre de femmes enceintes ayant reçu la 2ème dose de SP"
NUMBER_OF_PREGNANT_3RD_DOSE_SP = "Nbre de femmes enceintes ayant reçu 3 doses de SP"
NUMBER_OF_PREGNANT_4PLUS_DOSE_SP = "Nbre de femmes enceintes ayant reçu au moins 4 doses de SP"
LLINS_DISTRIBUTED_ANC = "Nombre de  MILDA distribuées au cours de la CPN"
LLINS_DISTRIBUTED_PEV = "Nombre de  MILDA distribuées au cours de PEV"

In [41]:
# variable names - ENGLISH
FACILITY_ID = "Facility ID"
CONSULTED_UNDER5_ONSITE = "Consulted, < 5, On Site"
CONSULTED_UNDER5_AGENT="Consulted, < 5, By Agent"
CONSULTED_UNDER5_DEAD="Consulted, < 5, Death"
CONSULTED_5ANDOVER_ONSITE = "Consulted, >= 5, On Site"
CONSULTED_5ANDOVER_AGENT="Consulted, >= 5, By Agent"
CONSULTED_5ANDOVER_DEAD="Consulted, >= 5, Death"
CONSULTED_PREGNANT_ONSITE = "Consulted, Pregnant, On Site"
CONSULTED_PREGNANT_DEAD="Consulted, Pregnant, Death"

SUSPECTED_MILD_UNDER5_MALE_ONSITE = "Suspected, Mild, Male, < 5 On Site"
SUSPECTED_MILD_UNDER5_FEMALE_ONSITE = "Suspected, Mild, Female, < 5 On Site"
SUSPECTED_SEVERE_UNDER5_MALE_ONSITE = "Suspected, Severe, Male, < 5 On Site"
SUSPECTED_SEVERE_UNDER5_FEMALE_ONSITE = "Suspected, Severe, Female, < 5 On Site"
SUSPECTED_MILD_UNDER5_MALE_AGENT = "Suspected, Mild, Male, < 5, By Agent"
SUSPECTED_MILD_UNDER5_FEMALE_AGENT = "Suspected, Mild, Female, < 5, By Agent"
SUSPECTED_MILD_PREGNANT_ONSITE = "Suspected, Mild, Pregnant, On Site"
SUSPECTED_SEVERE_PREGNANT_ONSITE = "Suspected, Severe, Pregnant, On Site"

SUSPECTED_MILD_5ANDOVER_MALE_ONSITE = "Suspected, Mild, Male, >= 5 On Site"
SUSPECTED_MILD_5ANDOVER_FEMALE_ONSITE = "Suspected, Mild, Female, >= 5 On Site"
SUSPECTED_SEVERE_5ANDOVER_MALE_ONSITE = "Suspected, Severe, Male, >= 5 On Site"
SUSPECTED_SEVERE_5ANDOVER_FEMALE_ONSITE = "Suspected, Severe, Female, >= 5 On Site"
SUSPECTED_MILD_5ANDOVER_MALE_AGENT = "Suspected, Mild, Male, >= 5, By Agent"
SUSPECTED_MILD_5ANDOVER_FEMALE_AGENT = "Suspected, Mild, Female, >= 5, By Agent"

TESTED_MICROSCOPY_UNDER5_ONSITE = "Tested, Microscopy, < 5, On Site"
TESTED_RDT_UNDER5_ONSITE = "Tested, RDT, < 5, On Site"
TESTED_RDT_UNDER5_AGENT = "Tested, RDT, < 5, By Agent"
TESTED_MICROSCOPY_5ANDOVER_ONSITE = "Tested, Microscopy, >= 5, On Site"
TESTED_RDT_5ANDOVER_ONSITE = "Tested, RDT, >= 5, On Site"
TESTED_RDT_5ANDOVER_AGENT = "Tested, RDT, >= 5, By Agent"
TESTED_MICROSCOPY_PREGNANT_ONSITE = "Tested, Microscopy, Pregnant, On Site"
TESTED_RDT_PREGNANT_ONSITE = "Tested, RDT, Pregnant, On Site"

CONFIRMED_MICROSCOPY_UNDER5_ONSITE = "Confirmed, Microscopy, < 5, On Site"
CONFIRMED_RDT_UNDER5_ONSITE = "Confirmed, RDT, < 5, On Site"
CONFIRMED_RDT_UNDER5_AGENT = "Confirmed, RDT, < 5, By Agent"
CONFIRMED_MICROSCOPY_5ANDOVER_ONSITE = "Confirmed, Microscopy, >= 5, On Site"
CONFIRMED_RDT_5ANDOVER_ONSITE = "Confirmed, RDT, >= 5, On Site"
CONFIRMED_RDT_5ANDOVER_AGENT = "Confirmed, RDT, >= 5, By Agent"
CONFIRMED_MICROSCOPY_PREGNANT_ONSITE = "Confirmed, Microscopy, Pregnant, On Site"
CONFIRMED_RDT_PREGNANT_ONSITE = "Confirmed, RDT, Pregnant, On Site"

TREATED_ACT_UNDER5_MALE_ONSITE = "Treated with ACT, Male, < 5, On Site"
TREATED_ACT_UNDER5_FEMALE_ONSITE = "Treated with ACT, Female, < 5, On Site"
TREATED_ACT_UNDER5_MALE_AGENT = "Treated with ACT, Male, < 5, By Agent"
TREATED_ACT_UNDER5_FEMALE_AGENT = "Treated with ACT, Female, < 5, By Agent"
TREATED_ACT_5ANDOVER_MALE_ONSITE = "Treated with ACT, Male, >= 5, On Site"
TREATED_ACT_5ANDOVER_FEMALE_ONSITE = "Treated with ACT, Female, >= 5, On Site"
TREATED_ACT_5ANDOVER_MALE_AGENT = "Treated with ACT, Male, >= 5, By Agent"
TREATED_ACT_5ANDOVER_FEMALE_AGENT = "Treated with ACT, Female, >= 5, By Agent"
TREATED_ACT_PREGNANT_ONSITE = "Treated with ACT, Pregnant, On Site"

TREATED_SEVERE_UNDER5_MALE_ONSITE = "Severe Cases Treated, Male, < 5, On Site"
TREATED_SEVERE_UNDER5_FEMALE_ONSITE = "Severe Cases Treated, Female, < 5, On Site"
TREATED_SEVERE_5ANDOVER_MALE_ONSITE = "Severe Cases Treated, Male, >= 5, On Site"
TREATED_SEVERE_5ANDOVER_FEMALE_ONSITE = "Severe Cases Treated, Female, >= 5, On Site"
TREATED_SEVERE_PREGNANT_ONSITE = "Severe Cases Treated, Pregnant, On Site"

REFERRED_UNDER5_ONSITE = "Cases Referred, < 5, On Site"
REFERRED_UNDER5_AGENT = "Cases Referred, < 5, By Agent"
REFERRED_5ANDOVER_ONSITE = "Cases Referred, >= 5, On Site"
REFERRED_5ANDOVER_AGENT = "Cases Referred, >= 5, By Agent"
REFERRED_PREGNANT_ONSITE = "Cases Referred, Pregnant, On Site"
REFERRED_PREGNANT_AGENT = "Cases Referred, Pregnant, By Agent"

NUMBER_OF_PREGNANT_WOMEN_1ST_ANC = "Total number of pregnant women who made the first ANC"
NUMBER_OF_PREGNANT_1ST_DOSE_SP = "Total number of pregnant women who received the first dose of SP"
NUMBER_OF_PREGNANT_2ND_DOSE_SP = "Total number of pregnant women who received the second dose of SP"
NUMBER_OF_PREGNANT_3RD_DOSE_SP = "Total number of pregnant women who received the third dose of SP"
NUMBER_OF_PREGNANT_4PLUS_DOSE_SP = "Total number of pregnant women who received at least 4 doses of SP"
LLINS_DISTRIBUTED_ANC = "Number of LLINs distributed during ANC"
LLINS_DISTRIBUTED_PEV = "Number of LLINs distributed during EPI"

In [6]:
# initialize record

def init_record():
    
    record = {}

    record[NUMBER_OF_PREGNANT_WOMEN_1ST_ANC] = None
    record[NUMBER_OF_PREGNANT_1ST_DOSE_SP] = None
    record[NUMBER_OF_PREGNANT_2ND_DOSE_SP] = None
    record[NUMBER_OF_PREGNANT_3RD_DOSE_SP] = None
    record[NUMBER_OF_PREGNANT_4PLUS_DOSE_SP] = None
    record[LLINS_DISTRIBUTED_ANC] = None
    record[LLINS_DISTRIBUTED_PEV] = None
    record[CONSULTED_UNDER5_ONSITE] = None
    record[SUSPECTED_MILD_UNDER5_MALE_ONSITE] = None
    record[SUSPECTED_MILD_UNDER5_FEMALE_ONSITE] = None
    record[SUSPECTED_SEVERE_UNDER5_MALE_ONSITE] = None
    record[SUSPECTED_SEVERE_UNDER5_FEMALE_ONSITE] = None 
    record[TESTED_MICROSCOPY_UNDER5_ONSITE] = None
    record[TESTED_RDT_UNDER5_ONSITE] = None
    record[CONFIRMED_MICROSCOPY_UNDER5_ONSITE] = None
    record[CONFIRMED_RDT_UNDER5_ONSITE] = None
    record[TREATED_ACT_UNDER5_MALE_ONSITE] = None
    record[TREATED_ACT_UNDER5_FEMALE_ONSITE] = None
    record[TREATED_SEVERE_UNDER5_MALE_ONSITE] = None
    record[TREATED_SEVERE_UNDER5_FEMALE_ONSITE] = None
    #record[REFERRED_UNDER5_ONSITE] = None
    record[CONSULTED_UNDER5_AGENT] = None
    record[SUSPECTED_MILD_UNDER5_MALE_AGENT] = None
    record[SUSPECTED_MILD_UNDER5_FEMALE_AGENT] = None
    record[TESTED_RDT_UNDER5_AGENT] = None
    record[CONFIRMED_RDT_UNDER5_AGENT] = None
    record[TREATED_ACT_UNDER5_MALE_AGENT] = None
    record[TREATED_ACT_UNDER5_FEMALE_AGENT] = None
    #record[REFERRED_UNDER5_AGENT] = None
    record[CONSULTED_UNDER5_DEAD] = None
    record[CONSULTED_5ANDOVER_ONSITE] = None
    record[SUSPECTED_MILD_5ANDOVER_MALE_ONSITE] = None
    record[SUSPECTED_MILD_5ANDOVER_FEMALE_ONSITE] = None
    record[SUSPECTED_SEVERE_5ANDOVER_MALE_ONSITE] = None
    record[SUSPECTED_SEVERE_5ANDOVER_FEMALE_ONSITE] = None
    record[TESTED_MICROSCOPY_5ANDOVER_ONSITE] = None
    record[TESTED_RDT_5ANDOVER_ONSITE] = None
    record[CONFIRMED_MICROSCOPY_5ANDOVER_ONSITE] = None
    record[CONFIRMED_RDT_5ANDOVER_ONSITE] = None
    record[TREATED_ACT_5ANDOVER_MALE_ONSITE] = None
    record[TREATED_ACT_5ANDOVER_FEMALE_ONSITE] = None
    record[TREATED_SEVERE_5ANDOVER_MALE_ONSITE] = None
    record[TREATED_SEVERE_5ANDOVER_FEMALE_ONSITE] = None
    #record[REFERRED_5ANDOVER_ONSITE] = None
    record[CONSULTED_5ANDOVER_AGENT] = None
    record[SUSPECTED_MILD_5ANDOVER_MALE_AGENT] = None
    record[SUSPECTED_MILD_5ANDOVER_FEMALE_AGENT] = None
    record[TESTED_RDT_5ANDOVER_AGENT] = None
    record[CONFIRMED_RDT_5ANDOVER_AGENT] = None
    record[TREATED_ACT_5ANDOVER_MALE_AGENT] = None
    record[TREATED_ACT_5ANDOVER_FEMALE_AGENT] = None
    #record[REFERRED_5ANDOVER_AGENT] = None
    record[CONSULTED_5ANDOVER_DEAD] = None
    record[CONSULTED_PREGNANT_ONSITE] = None
    record[SUSPECTED_MILD_PREGNANT_ONSITE] = None
    record[SUSPECTED_SEVERE_PREGNANT_ONSITE] = None
    record[TESTED_MICROSCOPY_PREGNANT_ONSITE] = None
    record[TESTED_RDT_PREGNANT_ONSITE] = None
    record[CONFIRMED_MICROSCOPY_PREGNANT_ONSITE] = None
    record[CONFIRMED_RDT_PREGNANT_ONSITE] = None
    record[TREATED_ACT_PREGNANT_ONSITE] = None
    record[TREATED_SEVERE_PREGNANT_ONSITE] = None
    #record[REFERRED_PREGNANT_ONSITE] = None
    #record[REFERRED_PREGNANT_AGENT] = None
    record[CONSULTED_PREGNANT_DEAD] = None
    
    return record

In [7]:

# Get this in the HMIS

# Cases Consulted
HMISUID_CONSULTED_UNDER5_ONSITE = "luipiCKVB47" 
HMISCO_CONSULTED_UNDER5_ONSITE = "vWneublYhOH"

HMISUID_CONSULTED_UNDER5_AGENT = "R9INGXgdKHN"
HMISCO_CONSULTED_UNDER5_AGENT = "vWneublYhOH"

HMISUID_CONSULTED_5ANDOVER_ONSITE = "luipiCKVB47" 
HMISCO_CONSULTED_5ANDOVER_ONSITE = "ldSKD4qkziO"

HMISUID_CONSULTED_5ANDOVER_AGENT = "R9INGXgdKHN"
HMISCO_CONSULTED_5ANDOVER_AGENT = "ldSKD4qkziO"

HMISUID_CONSULTED_PREGNANT_ONSITE = "E6T6Hl6vJxo"
HMISCO_CONSULTED_PREGNANT_AGENT = "HllvX50cXC0"

# Cases Suspected

## Under 5

HMISUID_SUSPECTED_MILD_UNDER5_MALE_ONSITE = "lOcvVwulcR0"
HMISCO_SUSPECTED_MILD_UNDER5_MALE_ONSITE = "bzOU7jTPugw"

HMISUID_SUSPECTED_MILD_UNDER5_FEMALE_ONSITE = "lOcvVwulcR0"
HMISCO_SUSPECTED_MILD_UNDER5_FEMALE_ONSITE = "gvsvZDDUEQv"

HMISUID_SUSPECTED_SEVERE_UNDER5_MALE_ONSITE = "lOcvVwulcR0"
HMISCO_SUSPECTED_SEVERE_UNDER5_MALE_ONSITE = "BYxQsKxVf37"

HMISUID_SUSPECTED_SEVERE_UNDER5_FEMALE_ONSITE = "lOcvVwulcR0"
HMISCO_SUSPECTED_SEVERE_UNDER5_FEMALE_ONSITE = "JN7CZ0uvdIg"

HMISUID_SUSPECTED_MILD_UNDER5_MALE_AGENT = "hEzTEvEy2Qo"
HMISCO_SUSPECTED_MILD_UNDER5_MALE_AGENT = "bzOU7jTPugw"

HMISUID_SUSPECTED_MILD_UNDER5_FEMALE_AGENT = "hEzTEvEy2Qo"
HMISCO_SUSPECTED_MILD_UNDER5_FEMALE_AGENT = "gvsvZDDUEQv"

HMISUID_SUSPECTED_MILD_PREGNANT_ONSITE = "n64UFGB7GN0"
HMISCO_SUSPECTED_MILD_PREGNANT_ONSITE = "XGqqCoXNbny"

## 5 and over

HMISUID_SUSPECTED_MILD_5ANDOVER_MALE_ONSITE = "lOcvVwulcR0"
HMISCO_SUSPECTED_MILD_5ANDOVER_MALE_ONSITE = "tfmFXouuZfd"

HMISUID_SUSPECTED_MILD_5ANDOVER_FEMALE_ONSITE = "lOcvVwulcR0"
HMISCO_SUSPECTED_MILD_5ANDOVER_FEMALE_ONSITE = "NiYuVFcZp5R"

HMISUID_SUSPECTED_SEVERE_5ANDOVER_MALE_ONSITE = "lOcvVwulcR0"
HMISCO_SUSPECTED_SEVERE_5ANDOVER_MALE_ONSITE = "fDfPdxodPKk"

HMISUID_SUSPECTED_SEVERE_5ANDOVER_FEMALE_ONSITE = "lOcvVwulcR0"
HMISCO_SUSPECTED_SEVERE_5ANDOVER_FEMALE_ONSITE = "xsOfSmBRzYD"

HMISUID_SUSPECTED_MILD_5ANDOVER_MALE_AGENT = "hEzTEvEy2Qo"
HMISCO_SUSPECTED_MILD_5ANDOVER_MALE_AGENT = "tfmFXouuZfd"

HMISUID_SUSPECTED_MILD_5ANDOVER_FEMALE_AGENT = "hEzTEvEy2Qo"
HMISCO_SUSPECTED_MILD_5ANDOVER_FEMALE_AGENT = "NiYuVFcZp5R"

HMISUID_SUSPECTED_SEVERE_PREGNANT_ONSITE = "n64UFGB7GN0"
HMISCO_SUSPECTED_SEVERE_PREGNANT_ONSITE = "fnIKAr0BqO2"


# Cases Tested

HMISUID_TESTED_MICROSCOPY_UNDER5_ONSITE = "EH1aOtaF9UH"
HMISUCO_TESTED_MICROSCOPY_UNDER5_ONSITE = "vSY3Jy8alDS"

HMISUID_TESTED_RDT_UNDER5_ONSITE = "EH1aOtaF9UH"
HMISCO_TESTED_RDT_UNDER5_ONSITE = "PCmjuGoIUzu"

HMISUID_TESTED_RDT_UNDER5_AGENT = "nNpjVjv5MPZ"
HMISCO_TESTED_RDT_UNDER5_AGENT = "PCmjuGoIUzu"

HMISUID_TESTED_MICROSCOPY_5ANDOVER_ONSITE = "EH1aOtaF9UH"
HMISCO_TESTED_MICROSCOPY_5ANDOVER_ONSITE = "DJxLwxLdJgm"

HMISUID_TESTED_RDT_5ANDOVER_ONSITE = "EH1aOtaF9UH"
HMISCO_TESTED_RDT_5ANDOVER_ONSITE = "CQlh6yjL5tQ"

HMISUID_TESTED_RDT_5ANDOVER_AGENT = "nNpjVjv5MPZ"
HMISCO_TESTED_RDT_5ANDOVER_AGENT = "CQlh6yjL5tQ"

HMISUID_TESTED_MICROSCOPY_PREGNANT_ONSITE = "jQqBsnxB6dG"
HMISCO_TESTED_MICROSCOPY_PREGNANT_ONSITE = "NSpjTE7qeb1"

HMISUID_TESTED_RDT_PREGNANT_ONSITE = "jQqBsnxB6dG"
HMISCO_TESTED_RDT_PREGNANT_ONSITE = "U7greWKtYoh"

# Cases Confirmed

HMISUID_CONFIRMED_MICROSCOPY_UNDER5_ONSITE = "Aps3ZQoFjs4"
HMISUCO_CONFIRMED_MICROSCOPY_UNDER5_ONSITE = "vSY3Jy8alDS"

HMISUID_CONFIRMED_RDT_UNDER5_ONSITE = "Aps3ZQoFjs4"
HMISUCO_CONFIRMED_RDT_UNDER5_ONSITE = "PCmjuGoIUzu"

HMISUID_CONFIRMED_RDT_UNDER5_AGENT = "jeeDrRedEkY"
HMISUCO_CONFIRMED_RDT_UNDER5_AGENT = "PCmjuGoIUzu"

HMISUID_CONFIRMED_MICROSCOPY_5ANDOVER_ONSITE = "Aps3ZQoFjs4"
HMISUCO_CONFIRMED_MICROSCOPY_5ANDOVER_ONSITE = "DJxLwxLdJgm"

HMISUID_CONFIRMED_RDT_5ANDOVER_ONSITE = "Aps3ZQoFjs4"
HMISUCO_CONFIRMED_RDT_5ANDOVER_ONSITE = "CQlh6yjL5tQ"

HMISUID_CONFIRMED_RDT_5ANDOVER_AGENT = "jeeDrRedEkY"
HMISUCO_CONFIRMED_RDT_5ANDOVER_AGENT = "CQlh6yjL5tQ"

HMISUID_CONFIRMED_MICROSCOPY_PREGNANT_ONSITE = "azpNCbgSjF3"
HMISUCO_CONFIRMED_MICROSCOPY_PREGNANT_ONSITE = "NSpjTE7qeb1"

HMISUID_CONFIRMED_RDT_PREGNANT_ONSITE = "azpNCbgSjF3"
HMISUCO_CONFIRMED_RDT_PREGNANT_ONSITE = "U7greWKtYoh"

# Cases Treated with ACT

HMISUID_TREATED_ACT_UNDER5_MALE_ONSITE = "JPUCOxp5uW0"
HMISUCO_TREATED_ACT_UNDER5_MALE_ONSITE = "QpF17oBBHtv"

HMISUID_TREATED_ACT_UNDER5_FEMALE_ONSITE = "JPUCOxp5uW0"
HMISUCO_TREATED_ACT_UNDER5_FEMALE_ONSITE = "InVOmW0IqKa"

HMISUID_TREATED_ACT_UNDER5_MALE_AGENT = "yVBqUvZYTWb"
HMISUCO_TREATED_ACT_UNDER5_MALE_AGENT = "QpF17oBBHtv"

HMISUID_TREATED_ACT_UNDER5_FEMALE_AGENT = "yVBqUvZYTWb"
HMISUCO_TREATED_ACT_UNDER5_FEMALE_AGENT = "InVOmW0IqKa"

HMISUID_TREATED_ACT_5ANDOVER_MALE_ONSITE = "JPUCOxp5uW0"
HMISUCO_TREATED_ACT_5ANDOVER_MALE_ONSITE = "aYPmc5mue8I"

HMISUID_TREATED_ACT_5ANDOVER_FEMALE_ONSITE = "JPUCOxp5uW0"
HMISUCO_TREATED_ACT_5ANDOVER_FEMALE_ONSITE = "H4Kkg704dwZ"

HMISUID_TREATED_ACT_5ANDOVER_MALE_AGENT = "yVBqUvZYTWb"
HMISUCO_TREATED_ACT_5ANDOVER_MALE_AGENT = "aYPmc5mue8I"

HMISUID_TREATED_ACT_5ANDOVER_FEMALE_AGENT = "yVBqUvZYTWb"
HMISUCO_TREATED_ACT_5ANDOVER_FEMALE_AGENT = "H4Kkg704dwZ"

HMISUID_TREATED_ACT_PREGNANT_ONSITE = "TLa3MtZmPXw"
HMISUCO_TREATED_ACT_PREGNANT_ONSITE = "HllvX50cXC0"

HMISUID_NUMBER_OF_WOMEN_1ST_ANC = "AXPaMiOmzVj"
HMISUCO_NUMBER_OF_WOMEN_1ST_ANC = "HllvX50cXC0"

HMISUID_NUMBER_OF_WOMEN_1ST_DOSE_SP = "IGNkgBmHt5R"
HMISUCO_NUMBER_OF_WOMEN_1ST_DOSE_SP = "HllvX50cXC0"

HMISUID_NUMBER_OF_WOMEN_2ND_DOSE_SP = "xLsD01hILMo"
HMISUCO_NUMBER_OF_WOMEN_2ND_DOSE_SP = "HllvX50cXC0"

HMISUID_NUMBER_OF_WOMEN_3RD_DOSE_SP = "puqUTGuDgPI"
HMISUCO_NUMBER_OF_WOMEN_3RD_DOSE_SP = "HllvX50cXC0"

HMISUID_NUMBER_OF_WOMEN_4PLUS_DOSE_SP = "jvHi0PqdwAR"
HMISUCO_NUMBER_OF_WOMEN_4PLUS_DOSE_SP = "HllvX50cXC0"

HMISUID_NUMBER_OF_LLIN_DIST_ANC = "XRbrFeJfnkg"
HMISUCO_NUMBER_OF_LLIN_DIST_ANC = "HllvX50cXC0"

HMISUID_NUMBER_OF_LLIN_DIST_EPI = "eGJ4FSuGkRE"
HMISUCO_NUMBER_OF_LLIN_DIST_EPI = "HllvX50cXC0"

#orgs_mapping[facility]

In [8]:
# pass in a json object. This will come from DHIS2
def build_facility_record_from_hmis(result, facility):
    
    hmis_export = []

    record = init_record()

    for org in result["dataValues"]:

        if org["orgUnit"] == facility and org["dataElement"] == HMISUID_CONSULTED_UNDER5_ONSITE  and org["categoryOptionCombo"] == HMISCO_CONSULTED_UNDER5_ONSITE:
            record[CONSULTED_UNDER5_ONSITE] = org["value"]

        if org["orgUnit"] == facility and org["dataElement"] == HMISUID_CONSULTED_UNDER5_AGENT  and org["categoryOptionCombo"] == HMISCO_CONSULTED_UNDER5_AGENT:
            record[CONSULTED_UNDER5_AGENT] = org["value"]

        if org["orgUnit"] == facility and org["dataElement"] == HMISUID_CONSULTED_5ANDOVER_ONSITE  and org["categoryOptionCombo"] == HMISCO_CONSULTED_5ANDOVER_ONSITE:
            record[CONSULTED_5ANDOVER_ONSITE] = org["value"]

        if org["orgUnit"] == facility and org["dataElement"] == HMISUID_CONSULTED_5ANDOVER_AGENT  and org["categoryOptionCombo"] == HMISCO_CONSULTED_5ANDOVER_AGENT:
            record[CONSULTED_5ANDOVER_AGENT] = org["value"]

        if org["orgUnit"] == facility and org["dataElement"] == HMISUID_CONSULTED_PREGNANT_ONSITE  and org["categoryOptionCombo"] == HMISCO_CONSULTED_PREGNANT_AGENT:
            record[CONSULTED_PREGNANT_ONSITE] = org["value"]

        if org["orgUnit"] == facility and org["dataElement"] == HMISUID_SUSPECTED_MILD_UNDER5_MALE_ONSITE  and org["categoryOptionCombo"] == HMISCO_SUSPECTED_MILD_UNDER5_MALE_ONSITE:
            record[SUSPECTED_MILD_UNDER5_MALE_ONSITE] = org["value"]

        if org["orgUnit"] == facility and org["dataElement"] == HMISUID_SUSPECTED_MILD_UNDER5_FEMALE_ONSITE  and org["categoryOptionCombo"] == HMISCO_SUSPECTED_MILD_UNDER5_FEMALE_ONSITE:
            record[SUSPECTED_MILD_UNDER5_FEMALE_ONSITE] = org["value"]

        if org["orgUnit"] == facility and org["dataElement"] == HMISUID_SUSPECTED_SEVERE_UNDER5_MALE_ONSITE  and org["categoryOptionCombo"] == HMISCO_SUSPECTED_SEVERE_UNDER5_MALE_ONSITE:
            record[SUSPECTED_SEVERE_UNDER5_MALE_ONSITE] = org["value"]

        if org["orgUnit"] == facility and org["dataElement"] == HMISUID_SUSPECTED_SEVERE_UNDER5_FEMALE_ONSITE  and org["categoryOptionCombo"] == HMISCO_SUSPECTED_SEVERE_UNDER5_FEMALE_ONSITE:
            record[SUSPECTED_SEVERE_UNDER5_FEMALE_ONSITE] = org["value"]

        if org["orgUnit"] == facility and org["dataElement"] == HMISUID_SUSPECTED_MILD_UNDER5_MALE_AGENT  and org["categoryOptionCombo"] == HMISCO_SUSPECTED_MILD_UNDER5_MALE_AGENT:
            record[SUSPECTED_MILD_UNDER5_MALE_AGENT] = org["value"]

        if org["orgUnit"] == facility and org["dataElement"] == HMISUID_SUSPECTED_MILD_UNDER5_FEMALE_AGENT  and org["categoryOptionCombo"] == HMISCO_SUSPECTED_MILD_UNDER5_FEMALE_AGENT:
            record[SUSPECTED_MILD_UNDER5_FEMALE_AGENT] = org["value"]

        if org["orgUnit"] == facility and org["dataElement"] == HMISUID_SUSPECTED_MILD_UNDER5_MALE_AGENT  and org["categoryOptionCombo"] == HMISCO_SUSPECTED_MILD_UNDER5_MALE_AGENT:
            record[SUSPECTED_MILD_UNDER5_MALE_AGENT] = org["value"]

        if org["orgUnit"] == facility and org["dataElement"] == HMISUID_SUSPECTED_MILD_UNDER5_FEMALE_AGENT  and org["categoryOptionCombo"] == HMISCO_SUSPECTED_MILD_UNDER5_FEMALE_AGENT:
            record[SUSPECTED_MILD_UNDER5_FEMALE_AGENT] = org["value"]

        if org["orgUnit"] == facility and org["dataElement"] == HMISUID_SUSPECTED_MILD_PREGNANT_ONSITE  and org["categoryOptionCombo"] == HMISCO_SUSPECTED_MILD_PREGNANT_ONSITE:
            record[SUSPECTED_MILD_PREGNANT_ONSITE] = org["value"]

        if org["orgUnit"] == facility and org["dataElement"] == HMISUID_SUSPECTED_MILD_5ANDOVER_MALE_ONSITE  and org["categoryOptionCombo"] == HMISCO_SUSPECTED_MILD_5ANDOVER_MALE_ONSITE:
            record[SUSPECTED_MILD_5ANDOVER_MALE_ONSITE] = org["value"]

        if org["orgUnit"] == facility and org["dataElement"] == HMISUID_SUSPECTED_MILD_5ANDOVER_FEMALE_ONSITE  and org["categoryOptionCombo"] == HMISCO_SUSPECTED_MILD_5ANDOVER_FEMALE_ONSITE:
            record[SUSPECTED_MILD_5ANDOVER_FEMALE_ONSITE] = org["value"]

        if org["orgUnit"] == facility and org["dataElement"] == HMISUID_SUSPECTED_SEVERE_5ANDOVER_MALE_ONSITE  and org["categoryOptionCombo"] == HMISCO_SUSPECTED_SEVERE_5ANDOVER_MALE_ONSITE:
            record[SUSPECTED_SEVERE_5ANDOVER_MALE_ONSITE] = org["value"]

        if org["orgUnit"] == facility and org["dataElement"] == HMISUID_SUSPECTED_SEVERE_5ANDOVER_FEMALE_ONSITE  and org["categoryOptionCombo"] == HMISCO_SUSPECTED_SEVERE_5ANDOVER_FEMALE_ONSITE:
            record[SUSPECTED_SEVERE_5ANDOVER_FEMALE_ONSITE] = org["value"]

        if org["orgUnit"] == facility and org["dataElement"] == HMISUID_SUSPECTED_MILD_5ANDOVER_MALE_AGENT  and org["categoryOptionCombo"] == HMISCO_SUSPECTED_MILD_5ANDOVER_MALE_AGENT:
            record[SUSPECTED_MILD_5ANDOVER_MALE_AGENT] = org["value"]

        if org["orgUnit"] == facility and org["dataElement"] == HMISUID_SUSPECTED_MILD_5ANDOVER_FEMALE_AGENT  and org["categoryOptionCombo"] == HMISCO_SUSPECTED_MILD_5ANDOVER_FEMALE_AGENT:
            record[SUSPECTED_MILD_5ANDOVER_FEMALE_AGENT] = org["value"]

        if org["orgUnit"] == facility and org["dataElement"] == HMISUID_SUSPECTED_SEVERE_PREGNANT_ONSITE  and org["categoryOptionCombo"] == HMISCO_SUSPECTED_SEVERE_PREGNANT_ONSITE:
            record[SUSPECTED_SEVERE_PREGNANT_ONSITE] = org["value"]

        if org["orgUnit"] == facility and org["dataElement"] == HMISUID_TESTED_MICROSCOPY_UNDER5_ONSITE  and org["categoryOptionCombo"] == HMISUCO_TESTED_MICROSCOPY_UNDER5_ONSITE:
            record[TESTED_MICROSCOPY_UNDER5_ONSITE] = org["value"]

        if org["orgUnit"] == facility and org["dataElement"] == HMISUID_TESTED_RDT_UNDER5_ONSITE  and org["categoryOptionCombo"] == HMISCO_TESTED_RDT_UNDER5_ONSITE:
            record[TESTED_RDT_UNDER5_ONSITE] = org["value"]

        if org["orgUnit"] == facility and org["dataElement"] == HMISUID_TESTED_RDT_UNDER5_AGENT  and org["categoryOptionCombo"] == HMISCO_TESTED_RDT_UNDER5_AGENT:
            record[TESTED_RDT_UNDER5_AGENT] = org["value"]

        if org["orgUnit"] == facility and org["dataElement"] == HMISUID_TESTED_MICROSCOPY_5ANDOVER_ONSITE  and org["categoryOptionCombo"] == HMISCO_TESTED_MICROSCOPY_5ANDOVER_ONSITE:
            record[TESTED_MICROSCOPY_5ANDOVER_ONSITE] = org["value"]

        if org["orgUnit"] == facility and org["dataElement"] == HMISUID_TESTED_RDT_5ANDOVER_ONSITE  and org["categoryOptionCombo"] == HMISCO_TESTED_RDT_5ANDOVER_ONSITE:
            record[TESTED_RDT_5ANDOVER_ONSITE] = org["value"]

        if org["orgUnit"] == facility and org["dataElement"] == HMISUID_TESTED_RDT_5ANDOVER_AGENT  and org["categoryOptionCombo"] == HMISCO_TESTED_RDT_5ANDOVER_AGENT:
            record[TESTED_RDT_5ANDOVER_AGENT] = org["value"]

        if org["orgUnit"] == facility and org["dataElement"] == HMISUID_TESTED_MICROSCOPY_PREGNANT_ONSITE  and org["categoryOptionCombo"] == HMISCO_TESTED_MICROSCOPY_PREGNANT_ONSITE:
            record[TESTED_MICROSCOPY_PREGNANT_ONSITE] = org["value"]

        if org["orgUnit"] == facility and org["dataElement"] == HMISUID_TESTED_RDT_PREGNANT_ONSITE  and org["categoryOptionCombo"] == HMISCO_TESTED_RDT_PREGNANT_ONSITE:
            record[TESTED_RDT_PREGNANT_ONSITE] = org["value"]

        if org["orgUnit"] == facility and org["dataElement"] == HMISUID_CONFIRMED_MICROSCOPY_UNDER5_ONSITE  and org["categoryOptionCombo"] == HMISUCO_CONFIRMED_MICROSCOPY_UNDER5_ONSITE:
            record[CONFIRMED_MICROSCOPY_UNDER5_ONSITE] = org["value"]

        if org["orgUnit"] == facility and org["dataElement"] == HMISUID_CONFIRMED_RDT_UNDER5_ONSITE  and org["categoryOptionCombo"] == HMISUCO_CONFIRMED_RDT_UNDER5_ONSITE:
            record[CONFIRMED_RDT_UNDER5_ONSITE] = org["value"]

        if org["orgUnit"] == facility and org["dataElement"] == HMISUID_CONFIRMED_RDT_UNDER5_AGENT  and org["categoryOptionCombo"] == HMISUCO_CONFIRMED_RDT_UNDER5_AGENT:
            record[CONFIRMED_RDT_UNDER5_AGENT] = org["value"]

        if org["orgUnit"] == facility and org["dataElement"] == HMISUID_CONFIRMED_MICROSCOPY_5ANDOVER_ONSITE  and org["categoryOptionCombo"] == HMISUCO_CONFIRMED_MICROSCOPY_5ANDOVER_ONSITE:
            record[CONFIRMED_MICROSCOPY_5ANDOVER_ONSITE] = org["value"]

        if org["orgUnit"] == facility and org["dataElement"] == HMISUID_CONFIRMED_RDT_5ANDOVER_ONSITE  and org["categoryOptionCombo"] == HMISUCO_CONFIRMED_RDT_5ANDOVER_ONSITE:
            record[CONFIRMED_RDT_5ANDOVER_ONSITE] = org["value"]

        if org["orgUnit"] == facility and org["dataElement"] == HMISUID_CONFIRMED_RDT_5ANDOVER_AGENT  and org["categoryOptionCombo"] == HMISUCO_CONFIRMED_RDT_5ANDOVER_AGENT:
            record[CONFIRMED_RDT_5ANDOVER_AGENT] = org["value"]

        if org["orgUnit"] == facility and org["dataElement"] == HMISUID_CONFIRMED_RDT_PREGNANT_ONSITE  and org["categoryOptionCombo"] == HMISUCO_CONFIRMED_MICROSCOPY_PREGNANT_ONSITE:
            record[CONFIRMED_MICROSCOPY_PREGNANT_ONSITE] = org["value"]

        if org["orgUnit"] == facility and org["dataElement"] == HMISUID_CONFIRMED_RDT_PREGNANT_ONSITE  and org["categoryOptionCombo"] == HMISUCO_CONFIRMED_RDT_PREGNANT_ONSITE:
            record[CONFIRMED_RDT_PREGNANT_ONSITE] = org["value"]

        if org["orgUnit"] == facility and org["dataElement"] == HMISUID_TREATED_ACT_UNDER5_MALE_ONSITE  and org["categoryOptionCombo"] == HMISUCO_TREATED_ACT_UNDER5_MALE_ONSITE:
            record[TREATED_ACT_UNDER5_MALE_ONSITE] = org["value"]

        if org["orgUnit"] == facility and org["dataElement"] == HMISUID_TREATED_ACT_UNDER5_FEMALE_ONSITE  and org["categoryOptionCombo"] == HMISUCO_TREATED_ACT_UNDER5_FEMALE_ONSITE:
            record[TREATED_ACT_UNDER5_FEMALE_ONSITE] = org["value"]

        if org["orgUnit"] == facility and org["dataElement"] == HMISUID_TREATED_ACT_UNDER5_MALE_AGENT  and org["categoryOptionCombo"] == HMISUCO_TREATED_ACT_UNDER5_MALE_AGENT:
            record[TREATED_ACT_UNDER5_MALE_AGENT] = org["value"]

        if org["orgUnit"] == facility and org["dataElement"] == HMISUID_TREATED_ACT_UNDER5_FEMALE_AGENT  and org["categoryOptionCombo"] == HMISUCO_TREATED_ACT_UNDER5_FEMALE_AGENT:
            record[TREATED_ACT_UNDER5_FEMALE_AGENT] = org["value"]

        if org["orgUnit"] == facility and org["dataElement"] == HMISUID_TREATED_ACT_5ANDOVER_MALE_ONSITE  and org["categoryOptionCombo"] == HMISUCO_TREATED_ACT_5ANDOVER_MALE_ONSITE:
            record[TREATED_ACT_5ANDOVER_MALE_ONSITE] = org["value"]

        if org["orgUnit"] == facility and org["dataElement"] == HMISUID_TREATED_ACT_5ANDOVER_FEMALE_ONSITE  and org["categoryOptionCombo"] == HMISUCO_TREATED_ACT_5ANDOVER_FEMALE_ONSITE:
            record[TREATED_ACT_5ANDOVER_FEMALE_ONSITE] = org["value"]

        if org["orgUnit"] == facility and org["dataElement"] == HMISUID_TREATED_ACT_5ANDOVER_MALE_AGENT  and org["categoryOptionCombo"] == HMISUCO_TREATED_ACT_5ANDOVER_FEMALE_AGENT:
            record[TREATED_ACT_5ANDOVER_MALE_AGENT] = org["value"]

        if org["orgUnit"] == facility and org["dataElement"] == HMISUID_TREATED_ACT_5ANDOVER_FEMALE_AGENT  and org["categoryOptionCombo"] == HMISUCO_TREATED_ACT_5ANDOVER_FEMALE_AGENT:
            record[TREATED_ACT_5ANDOVER_FEMALE_AGENT] = org["value"]

        if org["orgUnit"] == facility and org["dataElement"] == HMISUID_TREATED_ACT_PREGNANT_ONSITE  and org["categoryOptionCombo"] == HMISUCO_TREATED_ACT_PREGNANT_ONSITE:
            record[TREATED_ACT_PREGNANT_ONSITE] = org["value"]

        if org["orgUnit"] == facility and org["dataElement"] == HMISUID_NUMBER_OF_WOMEN_1ST_ANC  and org["categoryOptionCombo"] == HMISUCO_NUMBER_OF_WOMEN_1ST_ANC:
            record[NUMBER_OF_PREGNANT_WOMEN_1ST_ANC] = org["value"]

        if org["orgUnit"] == facility and org["dataElement"] == HMISUID_NUMBER_OF_WOMEN_1ST_DOSE_SP  and org["categoryOptionCombo"] == HMISUCO_NUMBER_OF_WOMEN_1ST_DOSE_SP:
            record[NUMBER_OF_PREGNANT_1ST_DOSE_SP] = org["value"]

        if org["orgUnit"] == facility and org["dataElement"] == HMISUID_NUMBER_OF_WOMEN_2ND_DOSE_SP  and org["categoryOptionCombo"] == HMISUCO_NUMBER_OF_WOMEN_2ND_DOSE_SP:
            record[NUMBER_OF_PREGNANT_2ND_DOSE_SP] = org["value"]

        if org["orgUnit"] == facility and org["dataElement"] == HMISUID_NUMBER_OF_WOMEN_3RD_DOSE_SP  and org["categoryOptionCombo"] == HMISUCO_NUMBER_OF_WOMEN_3RD_DOSE_SP:
            record[NUMBER_OF_PREGNANT_3RD_DOSE_SP] = org["value"]

        if org["orgUnit"] == facility and org["dataElement"] == HMISUID_NUMBER_OF_WOMEN_4PLUS_DOSE_SP  and org["categoryOptionCombo"] == HMISUCO_NUMBER_OF_WOMEN_4PLUS_DOSE_SP:
            record[NUMBER_OF_PREGNANT_4PLUS_DOSE_SP] = org["value"]

        if org["orgUnit"] == facility and org["dataElement"] == HMISUID_NUMBER_OF_LLIN_DIST_ANC  and org["categoryOptionCombo"] == HMISUCO_NUMBER_OF_LLIN_DIST_ANC:
            record[LLINS_DISTRIBUTED_ANC] = org["value"]

        if org["orgUnit"] == facility and org["dataElement"] == HMISUID_NUMBER_OF_LLIN_DIST_EPI  and org["categoryOptionCombo"] == HMISUCO_NUMBER_OF_LLIN_DIST_EPI:
            record[LLINS_DISTRIBUTED_PEV] = org["value"]


    hmis_export.append(record)
    return hmis_export


In [13]:
import psycopg2
import psycopg2.extras

# reporting period is YYYYMM. For example, 201804

def build_facility_record_from_imported_workbooks(reporting_period, facility_id):

    reporting_year = reporting_period[:4]
    reporting_month = int(reporting_period[4:6])

    wb_export = []
    conn = psycopg2.connect("dbname=stoppalu-db-dev host=localhost port=5433")
    cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)

    query = "SELECT wfmgr.\"group\", "+\
                    "wfmgr.registration_method, "+\
                    "wfmgr.total_patients_all_causes,"+\
                    "wfmr.health_facility_id, "+\
                    "wfmgr.suspect_simple_male, "+\
                    "wfmgr.suspect_simple_female,"+\
                    "wfmgr.suspect_severe_male,"+\
                    "wfmgr.suspect_severe_female, "+\
                    "wfmgr.tested_microscope, "+\
                    "wfmgr.tested_rdt, "+\
                    "wfmgr.confirmed_microscope, "+\
                    "wfmgr.confirmed_rdt, "+\
                    "wfmgr.treated_act_male, "+\
                    "wfmgr.treated_act_female, "+\
                    "wfmgr.treated_severe_male, "+\
                    "wfmgr.treated_severe_female, "+\
                    "wfmgr.total_referrals, "+\
                    "wfmr.num_pregnant_anc_tested, "+\
                    "wfmr.num_pregnant_first_dose_sp, "+\
                    "wfmr.num_pregnant_second_dose_sp, "+\
                    "wfmr.num_pregnant_three_doses_sp, "+\
                    "wfmr.num_pregnant_fourth_dose_sp, "+\
                    "wfmr.llin_dist_cpn, "+\
                    "wfmr.llin_dist_pev "+\
            "FROM workbooks "+\
                    "INNER JOIN workbook_files wf on workbooks.id = wf.workbook_id "+\
                    "INNER JOIN workbook_facility_monthly_reports wfmr on wf.id = wfmr.workbook_file_id "+\
                    "INNER JOIN workbook_facility_malaria_group_reports wfmgr on wfmr.id = wfmgr.workbook_facility_monthly_report_id "+\
            "WHERE reporting_year = "+str(reporting_year)+"  AND reporting_month = "+str(reporting_month)+"  AND wfmr.health_facility_id = "+str(facility_id)

    # period type 3 is monthly
    cur.execute(query)

    row = cur.fetchone()

    record = init_record()

    while row is not None:

        record[NUMBER_OF_PREGNANT_WOMEN_1ST_ANC] = row['num_pregnant_anc_tested']
        record[NUMBER_OF_PREGNANT_1ST_DOSE_SP] = row['num_pregnant_first_dose_sp']
        record[NUMBER_OF_PREGNANT_2ND_DOSE_SP] = row['num_pregnant_second_dose_sp']
        record[NUMBER_OF_PREGNANT_3RD_DOSE_SP] = row['num_pregnant_three_doses_sp']
        record[NUMBER_OF_PREGNANT_4PLUS_DOSE_SP] = row['num_pregnant_fourth_dose_sp']
        record[LLINS_DISTRIBUTED_ANC] = row['llin_dist_cpn']
        record[LLINS_DISTRIBUTED_PEV] = row['llin_dist_pev']


        if row['group'] == "below_five":
            if row['registration_method'] == "structure":
                record[CONSULTED_UNDER5_ONSITE] = row['total_patients_all_causes']
                record[SUSPECTED_MILD_UNDER5_MALE_ONSITE] = row['suspect_simple_male']
                record[SUSPECTED_MILD_UNDER5_FEMALE_ONSITE] = row['suspect_simple_female']
                record[SUSPECTED_SEVERE_UNDER5_MALE_ONSITE] = row['suspect_severe_male']
                record[SUSPECTED_SEVERE_UNDER5_FEMALE_ONSITE] = row['suspect_severe_female']  
                record[TESTED_MICROSCOPY_UNDER5_ONSITE] = row['tested_microscope']
                record[TESTED_RDT_UNDER5_ONSITE] = row['tested_rdt']
                record[CONFIRMED_MICROSCOPY_UNDER5_ONSITE] = row['confirmed_microscope']
                record[CONFIRMED_RDT_UNDER5_ONSITE] = row['confirmed_rdt']
                record[TREATED_ACT_UNDER5_MALE_ONSITE] = row['treated_act_male']
                record[TREATED_ACT_UNDER5_FEMALE_ONSITE] = row['treated_act_female']
                record[TREATED_SEVERE_UNDER5_MALE_ONSITE] = row['treated_severe_male']
                record[TREATED_SEVERE_UNDER5_FEMALE_ONSITE] = row['treated_severe_female']
                #record[REFERRED_UNDER5_ONSITE] = row['total_referrals']
            if row['registration_method'] == "agent":
                record[CONSULTED_UNDER5_AGENT] = row['total_patients_all_causes']
                record[SUSPECTED_MILD_UNDER5_MALE_AGENT] = row['suspect_simple_male']
                record[SUSPECTED_MILD_UNDER5_FEMALE_AGENT] = row['suspect_simple_female']
                record[TESTED_RDT_UNDER5_AGENT] = row['tested_rdt']
                record[CONFIRMED_RDT_UNDER5_AGENT] = row['confirmed_rdt']
                record[TREATED_ACT_UNDER5_MALE_AGENT] = row['treated_act_male']
                record[TREATED_ACT_UNDER5_FEMALE_AGENT] = row['treated_act_female']
                #record[REFERRED_UNDER5_AGENT] = row['total_referrals']
            if row['registration_method'] == "death":
                record[CONSULTED_UNDER5_DEAD] = row['total_patients_all_causes']

        if row['group'] == "five_and_above":
            if row['registration_method'] == "structure":
                record[CONSULTED_5ANDOVER_ONSITE] = row['total_patients_all_causes']
                record[SUSPECTED_MILD_5ANDOVER_MALE_ONSITE] = row['suspect_simple_male']
                record[SUSPECTED_MILD_5ANDOVER_FEMALE_ONSITE] = row['suspect_simple_female']
                record[SUSPECTED_SEVERE_5ANDOVER_MALE_ONSITE] = row['suspect_severe_male']
                record[SUSPECTED_SEVERE_5ANDOVER_FEMALE_ONSITE] = row['suspect_severe_female']
                record[TESTED_MICROSCOPY_5ANDOVER_ONSITE] = row['tested_microscope']
                record[TESTED_RDT_5ANDOVER_ONSITE] = row['tested_rdt']
                record[CONFIRMED_MICROSCOPY_5ANDOVER_ONSITE] = row['confirmed_microscope']
                record[CONFIRMED_RDT_5ANDOVER_ONSITE] = row['confirmed_rdt']
                record[TREATED_ACT_5ANDOVER_MALE_ONSITE] = row['treated_act_male']
                record[TREATED_ACT_5ANDOVER_FEMALE_ONSITE] = row['treated_act_female']
                record[TREATED_SEVERE_5ANDOVER_MALE_ONSITE] = row['treated_severe_male']
                record[TREATED_SEVERE_5ANDOVER_FEMALE_ONSITE] = row['treated_severe_female']
                #record[REFERRED_5ANDOVER_ONSITE] = row['total_referrals']
            if row['registration_method'] == "agent":
                record[CONSULTED_5ANDOVER_AGENT] = row['total_patients_all_causes']
                record[SUSPECTED_MILD_5ANDOVER_MALE_AGENT] = row['suspect_simple_male']
                record[SUSPECTED_MILD_5ANDOVER_FEMALE_AGENT] = row['suspect_simple_female']
                record[TESTED_RDT_5ANDOVER_AGENT] = row['tested_rdt']
                record[CONFIRMED_RDT_5ANDOVER_AGENT] = row['confirmed_rdt']
                record[TREATED_ACT_5ANDOVER_MALE_AGENT] = row['treated_act_male']
                record[TREATED_ACT_5ANDOVER_FEMALE_AGENT] = row['treated_act_female']
                #record[REFERRED_5ANDOVER_AGENT] = row['total_referrals']
            if row['registration_method'] == "death":
                record[CONSULTED_5ANDOVER_DEAD] = row['total_patients_all_causes']

        if row['group'] == "pregnant":
            if row['registration_method'] == "structure":
                record[CONSULTED_PREGNANT_ONSITE] = row['total_patients_all_causes']
                record[SUSPECTED_MILD_PREGNANT_ONSITE] = row['suspect_simple_female']
                record[SUSPECTED_SEVERE_PREGNANT_ONSITE] = row['suspect_severe_female']
                record[TESTED_MICROSCOPY_PREGNANT_ONSITE] = row['tested_microscope']
                record[TESTED_RDT_PREGNANT_ONSITE] = row['tested_rdt']
                record[CONFIRMED_MICROSCOPY_PREGNANT_ONSITE] = row['confirmed_microscope']
                record[CONFIRMED_RDT_PREGNANT_ONSITE] = row['confirmed_rdt']
                record[TREATED_ACT_PREGNANT_ONSITE] = row['treated_act_female']
                record[TREATED_SEVERE_PREGNANT_ONSITE] = row['treated_severe_female']
                #record[REFERRED_PREGNANT_ONSITE] = row['total_referrals']
            if row['registration_method'] == "agent":
                #record[REFERRED_PREGNANT_AGENT] = row['total_referrals']
                pass
            if row['registration_method'] == "death":
                record[CONSULTED_PREGNANT_DEAD] = row['total_patients_all_causes']

        row = cur.fetchone()

    wb_export.append(record)
    
    cur.close()
    conn.close()
    
    
    return wb_export

##  Step 1: Export Data from the HMIS

In [86]:
import os
# obv you have to set these
hmis_user = os.environ['GUINEA_HMIS_USER']
hmis_pass = os.environ['GUINEA_HMIS_PASS']

reporting_period = "201902" # This is what you should adjust

hmis_url = "https://dhis2.sante.gov.gn/dhis/api/26/dataValueSets.json?"+ \
            "dataSet=TL7xWT7zMNN&period="+reporting_period+"&orgUnitGroup=UF9kVV05HJ0"

r = requests.get(hmis_url, auth=(hmis_user, hmis_pass))
result = r.json()

##  Step 2: Get Data Frames for each facility and compare them

In [88]:
count = 0

frames = []
not_reported = []

for org in orgs_mapping:
           
    #if org['hmisUID'] != "LSdejPq33sK":
       # continue
        
    facility_id = int(org['projectUID'].replace("FAC",""))
    
    # Build the HMIS Dataframe
    hmis_facility_record = build_facility_record_from_hmis(result, org['hmisUID'])
    hmis_df = pd.DataFrame(hmis_facility_record, dtype=np.int64)
    
    # Build the workbook dataframe
    wb_facility_record = build_facility_record_from_imported_workbooks(reporting_period, facility_id)
    workbook_df = pd.DataFrame(wb_facility_record, dtype=np.int64)
    
    # keep track of those facilities that didn't report at all
    if hmis_df.dropna(how='all').empty and workbook_df.dropna(how='all').empty:
        not_reported.append(org)
    
    # Comare the two
    diff_bool = (hmis_df != workbook_df).stack()  # Create Frame of comparison booleans
    diff_df = pd.concat([hmis_df.stack()[diff_bool], workbook_df.stack()[diff_bool]], axis=1)
    
    # write out the results to a cvs file
    diff_df.columns=["HMIS", "Workbook"]
    diff_df.drop(diff_df[(diff_df.HMIS.isnull()) & (diff_df.Workbook == 0)].index, inplace=True)

    diff_df['Facility'] = pd.Series(org['facility'], index=diff_df.index)
    diff_df['District'] = pd.Series(org['district'], index=diff_df.index)
    diff_df['Region'] = pd.Series(org['region'], index=diff_df.index)

    diff_df = diff_df.reset_index()
    diff_df.drop(['level_0'], axis=1, inplace=True)
    diff_df.rename(columns = {'level_1':'Indicator'}, inplace = True)
    frames.append(diff_df)
     
    count = count + 1 
    
final_df = pd.concat(frames)
final_df.to_csv(str(reporting_period)+'-dataset-comparison-report.csv',index=False)

In [89]:
workbook_df.shape

(1, 59)

## Here are the facilities that didn't report anything, anywhere
put these in the monthly report (in the second tab)

In [90]:
not_reported

[{'district': 'Matoto',
  'facility': 'Accar Roger',
  'hmisUID': 'GyQxDH68966',
  'projectUID': 'FAC00000226',
  'region': 'Conakry'},
 {'district': 'Matoto',
  'facility': 'Aammes Guinee',
  'hmisUID': 'XsJiIYtIUBj',
  'projectUID': 'FAC00000211',
  'region': 'Conakry'},
 {'district': 'Forecariyah',
  'facility': 'Bassia',
  'hmisUID': 'jLzJEzigj19',
  'projectUID': 'FAC00000062',
  'region': 'Kinda'}]